In [1]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

/opt/conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
#フォルダutilsにあるdataloader_image_classification.pyをインポート
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

#アリとハチの画像のパスを格納したリストを作成
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

#Datasetを作成
size = 224
mean = (0.485, 0.456, 0.406) #ImageNetのデータの平均値
std = (0.229, 0.224, 0.225) #ImageNetのデータの標準偏差
train_dataset = HymenopteraDataset(file_list = train_list, transform = ImageTransform(size, mean, std), phase="train")
val_dataset = HymenopteraDataset(file_list = val_list, transform = ImageTransform(size, mean, std), phase="val")

#DataLoaderを作成
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#辞書型オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [4]:
#学習済みのVGG16モデルをロード

#VGG16のインスタンスを作成
use_pretrained = True #学習済みのパラメータを使用
net = models.vgg16(pretrained = use_pretrained)

#VGG16の最後の出力層の出力ユニット数をアリとハチの2つに変更
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

#訓練モードに設定
net.train()

print("ネットワーク設定完了：学習済みの重みをロードし、訓練モードに変更しました。")



/opt/conda/envs/py38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/py38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ネットワーク設定完了：学習済みの重みをロードし、訓練モードに変更しました。


In [5]:
#損失関数の設定
criterion = nn.CrossEntropyLoss()

In [6]:
#VGG16のモデル構成を表示
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
#ファインチューニングで学習させるパラメーターを、変数params_to_updateの1~3に格納する
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

#学習させるパラメーター名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

#パラメータごとに書くリストに格納する
print("-----学習させるパラメーターと勾配計算の有無の確認-----")
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)
    
    else:
        param.requires_grad = False
        print("勾配計算なし：", name)
    

-----学習させるパラメーターと勾配計算の有無の確認-----
params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features

In [8]:
#最適化手法の設定
optimizer = optim.SGD([
    {"params": params_to_update_1, "lr": 1e-4},
    {"params": params_to_update_2, "lr": 5e-4},
    {"params": params_to_update_3, "lr": 1e-3}
], momentum=0.9)

In [9]:
#モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    #初期設定
    #GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス:", device)

    #ネットワークをGPUへ
    net.to(device)

    #ネットワークがある程度固定であれば、高速化する
    torch.backends.cudnn.benchmark = True

    #epochのループ
    for epoch in range(num_epochs):
        print ("Epoch {}/{}".format(epoch+1, num_epochs))
        print("-------------------")

        #epochごとの訓練と検証のループ
        for phase in ["train", "val"]:
            if phase == "train":
                net.train() #モデルを訓練モードに
            else:
                net.eval() #モデルを検証モードに
        
            epoch_loss = 0.0 #epochの損失和
            epoch_corrects = 0 #epochの正解数

            #未学習時の性能検証のため、epoch=0の時は検証を行わない
            if (epoch == 0) and (phase == "train"):
                continue

            #データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                #GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                #optimizerを初期化
                optimizer.zero_grad()
                
                #順伝搬（forward）計算
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) #損失を計算
                    _, preds = torch.max(outputs, 1) #ラベルを予測
                
                    #訓練時はバックプロパゲーション
                    if phase == "train":
                        loss.backward() #勾配を計算
                        optimizer.step() #パラメータを更新
                    
                    #結果の計算
                    epoch_loss += loss.item() * inputs.size(0) #lossの合計を更新
                    #正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

                #epochごとのlossと正解率の表示
                epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

                print("{} Loss: {:.4f} Acc {:.4f}".format(phase, epoch_loss, epoch_acc))

In [10]:
#学習・検証を実行する
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス: cpu
Epoch 1/2
-------------------


 20%|██        | 1/5 [00:04<00:17,  4.27s/it]

val Loss: 0.1672 Acc 0.0784


 40%|████      | 2/5 [00:08<00:12,  4.18s/it]

val Loss: 0.1707 Acc 0.1699


...

val Loss: 0.0265 Acc 0.6078


 80%|████████  | 4/5 [00:17<00:04,  4.45s/it]

val Loss: 0.0376 Acc 0.8105


100%|██████████| 5/5 [00:21<00:00,  4.30s/it]

val Loss: 0.0480 Acc 0.9608


In [11]:
#PyTorchのネットワークパラメータの保存
save_path = "./weights_fine_tuning.pth"
torch.save(net.state_dict, save_path)

In [16]:
#PyTorchのネットワークパラメータのロード
load_path = "./weights_fine_tuning.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

#GPU上で保存された重みをcpu上でロードする場合
load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
net.load_state_dict(load_weights)

TypeError: Expected state_dict to be dict-like, got <class 'method'>.